In [1]:
from evo.notebooks import ServiceManagerWidget

cache_location = "data"
input_path = f"{cache_location}/input"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

client_id = "daves-evo-client"
redirect_url = "http://localhost:32369/auth/callback"

manager = await ServiceManagerWidget.with_auth_code(
    discovery_url="https://discover.api.seequent.com",
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

c:\Users\david.knight\Downloads\forks\evo-data-converters\samples\duf-jupyter-conversion\.venv\Lib\site-packages\evo\notebooks\authorizer.py:108: UserWarning: The evo.notebooks.AuthorizationCodeAuthorizer is not secure, and should only ever be used in Jupyter notebooks in a private environment.
  warnings.warn(


ServiceManagerWidget(children=(VBox(children=(HBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR…

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import tkinter as tk
from tkinter import filedialog
import os
from pathlib import Path
from pyproj import CRS
from pyproj.database import get_codes

# Variable to store selected file path
selected_file_path = None

# Helper: read all env vars from file
def read_env_vars(env_path: Path):
    env = {}
    if env_path.exists():
        try:
            with open(env_path, 'r') as f:
                for line in f:
                    line = line.strip()
                    if not line or '=' not in line:
                        continue
                    k, v = line.split('=', 1)
                    env[k] = v.strip().strip('"')
        except Exception as e:
            print(f'ERROR: Unable to read .env: {e}')
    return env

# Helper: write/update single env var (keep unique keys)
def update_env_var(env_path: Path, key: str, value: str):
    lines = []
    if env_path.exists():
        with open(env_path, 'r') as f:
            for line in f:
                if not line.startswith(f'{key}='):
                    lines.append(line)
    lines.append(f'{key}={value}\n')
    with open(env_path, 'w') as f:
        f.writelines(lines)

# Preload saved selection and inputs from .env (if present)
env_file_path = Path(cache_location) / '.env'
os.makedirs(cache_location, exist_ok=True)

env_vars = read_env_vars(env_file_path)
saved_path = env_vars.get('SELECTED_DUF_FILE')

# Create widgets (neutral ipywidgets styling)
select_button = widgets.Button(
    description='Select DUF File',
    tooltip='Click to select a .duf file',
)
output_label = widgets.Label(value='No file selected')
error_label = widgets.Label(value='', style={'text_color': 'red'})

# EPSG code input widget
epsg_input = widgets.Text(
    description='EPSG code',
    placeholder='Enter EPSG code (e.g., 2193)',
    style={'description_width': '80px'},
)
epsg_info = widgets.Label(value='Enter EPSG code and press Enter to validate')

def validate_epsg(change):
    """Validate EPSG code using pyproj"""
    code = change['new'].strip() if isinstance(change, dict) else change.value.strip()
    
    if not code:
        epsg_info.value = 'Enter EPSG code'
        epsg_info.style = {}
        return
    
    epsg_info.value = 'Validating...'
    
    try:
        # Try to create CRS from EPSG code
        crs = CRS.from_epsg(int(code))
        epsg_info.value = f'Valid: {crs.name}'
        epsg_info.style = {'text_color': 'green'}
        update_env_var(env_file_path, 'EPSG_CODE', code)
    except ValueError:
        epsg_info.value = 'Invalid: EPSG code must be a number'
        epsg_info.style = {'text_color': 'red'}
    except Exception as e:
        epsg_info.value = f'Invalid: EPSG:{code} not found'
        epsg_info.style = {'text_color': 'red'}

epsg_input.on_submit(validate_epsg)
epsg_input.observe(validate_epsg, names='value')

object_path_input = widgets.Text(
    description='Object path',
    placeholder='Object path (optional)',
    style={'description_width': '80px'},
)
advanced_box = widgets.VBox([epsg_input, epsg_info, object_path_input])
advanced_box.layout.display = 'none'

# Apply preload state
if saved_path:
    p = Path(saved_path)
    if p.suffix.lower() != '.duf':
        error_label.value = 'ERROR: Saved file is not a .duf file'
    elif p.exists():
        selected_file_path = str(p)
        output_label.value = f'Selected: {p.name}'
        advanced_box.layout.display = ''  # show inputs
        # Preload EPSG and Object Path if present
        saved_epsg = env_vars.get('EPSG_CODE', '')
        if saved_epsg:
            epsg_input.value = saved_epsg
            # Auto-validate on load
            try:
                crs = CRS.from_epsg(int(saved_epsg))
                epsg_info.value = f'Valid: {crs.name}'
                epsg_info.style = {'text_color': 'green'}
            except:
                epsg_info.value = f'Invalid: EPSG:{saved_epsg} not found'
                epsg_info.style = {'text_color': 'red'}
        object_path_input.value = env_vars.get('OBJECT_PATH', '')
    else:
        error_label.value = 'ERROR: Saved file not found on disk'


def on_button_click(b):
    """Handle button click to open file dialog"""
    global selected_file_path
    
    error_label.value = ''
    output_label.value = 'Opening file dialog...'
    
    # Create file dialog
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    root.attributes('-topmost', True)  # Bring dialog to front
    
    file_path = filedialog.askopenfilename(
        title='Select DUF File',
        filetypes=[('DUF Files', '*.duf'), ('All Files', '*.*')]
    )
    
    root.destroy()
    
    if not file_path:
        output_label.value = 'No file selected'
        advanced_box.layout.display = 'none'
        return
    
    file_path = Path(file_path)
    
    # Validate file extension
    if file_path.suffix.lower() != '.duf':
        error_label.value = 'ERROR: Invalid file type. Only .duf files are allowed.'
        output_label.value = 'No file selected'
        advanced_box.layout.display = 'none'
        return
    
    # Store the selected file path
    selected_file_path = str(file_path)
    
    # Update .env with unique entry
    update_env_var(env_file_path, 'SELECTED_DUF_FILE', selected_file_path)
    
    # Display success and show inputs
    output_label.value = f'Selected: {file_path.name}'
    advanced_box.layout.display = ''
    print(f'Full path: {selected_file_path}')
    print(f'Updated {env_file_path} with unique SELECTED_DUF_FILE entry')

# Persist input changes to .env

def on_object_path_change(change):
    update_env_var(env_file_path, 'OBJECT_PATH', change['new'] or '')

# Attach event handlers
select_button.on_click(on_button_click)
object_path_input.observe(on_object_path_change, names='value')

# Display the widget
ui = widgets.VBox([
    select_button,
    output_label,
    error_label,
    advanced_box,
])
display(ui)


C:\Users\david.knight\AppData\Local\Temp\ipykernel_18472\3238363069.py:88: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  epsg_input.on_submit(validate_epsg)
